In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import numpy as np
import pandas as pd
import deepchem as dc
import os

/usr/local/anaconda3/envs/li_place/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Active learning of molecular conformations

This notebook is an idealized example of how active learning can be used to predict properties of molecules and polymers and guide experiments or computations.

### Quick summary of the method
- Load initial conformer data file
- Train Gaussian process regression (GPR) model on initial data
- Load next conformer data file
- Make a prediction on the new data
- Select new conformers, based on their uncertainty, to have their energy calculated and incorporated in the training set (In reality, I already calculated the engeries using UFF)
- Continue this loop until you get to a metric or uncertainly threshold of your choosing, or you hit a wall with your model :)

## Energy values are in kcal/mol

## Function Definitions

These functions are to help keep the active learning loop clean

In [4]:
def load(file):
    # load and featurize (CoulombMatrixEig) molecules from sdf file using deepchem  
    task = ['energy']
    smiles_field = "smiles"
    mol_field = "mol"
    featurizer_func = dc.feat.coulomb_matrices.CoulombMatrixEig(max_atoms=20, remove_hydrogens=False)
    loader = dc.data.SDFLoader(tasks=task, featurizer=featurizer_func, 
                               smiles_field="smiles", mol_field="mol")
    dataset = loader.featurize(file)
    return dataset.X, dataset.y

In [5]:
def load_and_split(file, seed=None):
    # load, featurize molecules from sdf file, and split using deepchem 
    task = ['energy']
    smiles_field = "smiles"
    mol_field = "mol"
    featurizer_func = dc.feat.coulomb_matrices.CoulombMatrixEig(max_atoms=20, remove_hydrogens=False)
    loader = dc.data.SDFLoader(tasks=task, featurizer=featurizer_func, 
                               smiles_field="smiles", mol_field="mol")
    dataset = loader.featurize(file)
    # randomly split data 
    # not optimizing hyperparameters so no validation split
    random_splitter = dc.splits.RandomSplitter()
    train_dataset, test_dataset = random_splitter.train_test_split(dataset, seed=seed)
    return train_dataset.X, train_dataset.y, test_dataset.X, test_dataset.y

In [6]:
def train_and_predict(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    # predict and get train rmse ... should be very small
    y_train_pred, y_train_pred_std = model.predict(X_train, return_std=True)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    # predict and get test rmse
    y_pred, y_pred_std = model.predict(X_test, return_std=True)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return np.round(train_rmse, decimals=2), np.round(test_rmse, decimals=2)

In [7]:
def predict_and_select(model, X_new_data, select_frac):
    # predict
    y_pred, y_pred_std = model.predict(X_new_data, return_std=True)
    # find the conformers with the highest prediction uncertainty
    uncert_select = np.argsort(y_pred_std)
    num_select = int(np.round(len(uncert_select) * (1.0 - select_frac)))
    # np.argsort sorts from min to max so selecting from the end of array gives the
    # the max uncertainty
    return uncert_select[num_select:]

In [8]:
def merge_and_split(select_idx, X_new, y_new, X_train, y_train, X_test, y_test, seed=None):
    X_uncert = X_new[select_idx]
    y_uncert = y_new[select_idx]
    X_tot = np.concatenate((X_train, X_test, X_uncert), axis=0)
    y_tot = np.concatenate((y_train, y_test, y_uncert), axis=0)
    # order is different than deepchem (train, train, test, test)
    X1_train, X1_test, y1_train, y1_test = train_test_split(X_tot, y_tot, test_size=0.2, 
                                                            random_state=seed)
    return X1_train, y1_train, X1_test, y1_test

## Collect all the data files

In [9]:
current_dir = os.getcwd()
dataset_dir = os.path.join(current_dir, "..", "data", "medium_dataset")
data_files = []
for file in os.listdir(dataset_dir):
    if file.endswith(".sdf"):
        data_files.append(os.path.join(dataset_dir, file))

## Active learning loop

This loop is variable (meaning you get a different answer each time you run it) because the splits are random. 

In [10]:
# seeds to initialize a deterministic state, needs to be the same length as the loop

In [11]:
seeds = [5, 22, 99]

In [12]:
# if you want random seeds set all values in seeds to None or remove seed=seeds[i] from loop

In [13]:
# TODO: random splitting changes the RMSE values a lot, 
# try stratified splitting to maintain underlying distributions

In [14]:
gpr_model = GaussianProcessRegressor(normalize_y=True)
for i, file in enumerate(data_files):
    # first iteration of the learning loop
    if i == 0:
        # load and split the initial data file
        X_train, y_train, X_test, y_test = load_and_split(file, seed=seeds[i])
        # initial model training and prediction
        train_rmse, test_rmse = train_and_predict(gpr_model, X_train, y_train, X_test, y_test)
        # print
        print("\n")
        print("Number of Training Data Points: " + str(len(X_train)))
        print("Initial Train RMSE: ".format(i=i) + str(train_rmse))
        print("Initial Test RMSE: ".format(i=i) + str(test_rmse))
        print("\n")
        
    # all subsequent iterations of learning loop
    else:
        # load new data
        X_new, y_new = load(file)
        # predict and select the most uncertain conformers
        select_idx = predict_and_select(gpr_model, X_new, 0.6)
        # merge the new data into the dataset
        X_train, y_train, X_test, y_test = merge_and_split(select_idx, X_new, y_new, 
                                                           X_train, y_train, 
                                                           X_test, y_test, seed=seeds[i])
        # train and predict
        train_rmse_update, test_rmse_update = train_and_predict(gpr_model, 
                                                                X_train, 
                                                                y_train, 
                                                                X_test, 
                                                                y_test)
        # print
        print("\n")
        print("Number of Training Data Points: " + str(len(X_train)))
        print("Loop {i} Train RMSE: ".format(i=i) + str(train_rmse_update))
        print("Loop {i} Test RMSE: ".format(i=i) + str(test_rmse_update))
        print("\n")

Loading raw samples now.
shard_size: 8192
Reading structures from /Users/bwood/google_drive/Documents/Projects/lithium_placement_ML/code/molecular_conformations_ML/notebooks/../data/medium_dataset/pe_conformer_1.sdf.
Currently featurizing feature_type: CoulombMatrixEig
Featurizing sample 0
TIMING: featurizing shard 0 took 0.450 s
TIMING: dataset construction took 0.600 s
Loading dataset from disk.
TIMING: dataset construction took 0.010 s
Loading dataset from disk.
TIMING: dataset construction took 0.007 s
Loading dataset from disk.


Number of Training Data Points: 400
Initial Train RMSE: 0.0
Initial Test RMSE: 21.42


Loading raw samples now.
shard_size: 8192
Reading structures from /Users/bwood/google_drive/Documents/Projects/lithium_placement_ML/code/molecular_conformations_ML/notebooks/../data/medium_dataset/pe_conformer_2.sdf.
Currently featurizing feature_type: CoulombMatrixEig
Featurizing sample 0
TIMING: featurizing shard 0 took 0.459 s
TIMING: dataset construction took 0.607 